In [1]:
import numpy as np
n_features = 20
term_pred = 'short_term'
data_file_name = 'data_y10_top_20_var'
endpt = 10;
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
l2_final=0.1
# work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'
# work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'
work_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2'

In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
# snippets_dir = work_dir+ '/code/snippet'
# snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
snippets_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/Git/Python_code/snippets'

import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,time,event,C08DIBAG_YEAR,C02AVGDI,DPMAINT,C08OTKAG_YEAR,C11MHAGE,CL1NTRIG,C02AVGSY,...,C02POP,C08NEPAG_YEAR,C08KYSAG_YEAR,DPMEINT,MMLAD,C35BP,C09SMKYR,MMLVDS,MMLVSSFS,DPVAE
0,100016012504,3689.025,0,0,72.0,0.05,0,41,147.0,114.0,...,120.0,0,0,0.150000,3.970000,3,12,4.010000,6.030000,0.450000
1,100023004268,3689.025,0,0,64.0,0.04,0,99,89.0,94.0,...,110.0,0,0,0.130000,3.540000,2,8,3.200000,3.300000,0.600000
2,100033323702,3689.025,0,0,73.0,0.05,0,99,110.0,105.0,...,110.0,0,0,0.160000,3.810000,2,0,3.600000,4.000000,0.490000
3,100048415270,1169.000,0,0,68.0,0.06,0,99,49.0,93.0,...,90.0,18,0,0.140000,3.320000,2,13,3.180000,3.960000,0.450000
4,100056526386,3689.025,0,0,76.0,0.07,0,99,91.0,118.0,...,116.0,0,0,0.180000,4.390000,2,0,2.800000,3.270000,0.590000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,3689.025,0,0,63.0,0.03,0,58,84.0,98.0,...,100.0,0,0,0.150000,3.430000,1,0,3.410000,3.730000,0.370000
4310,416771521620,3689.025,0,0,82.0,0.04,0,99,38.0,111.0,...,110.0,0,0,0.100000,3.130000,2,0,3.110000,3.200000,0.830000
4311,416783315386,3689.025,0,0,64.0,0.04,0,99,47.0,109.0,...,100.0,0,0,0.127864,3.415775,2,3,3.162774,3.230787,0.588047
4312,416796224310,3689.025,0,0,69.0,0.04,0,99,40.0,108.0,...,100.0,0,0,0.080000,4.640000,2,12,3.260000,4.670000,0.790000


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/all_training_ID_outerloop_cohort_0_10.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,100016012504,0,3689.025,-0.092185,0.270520,0.157052,-0.089885,-5.481100,0.928896,0.535854,...,1.421847,-0.061243,-0.090552,0.748108,0.953969,1.587715,1.311295,1.878146,1.230919,-0.939192
1,100023004268,0,3689.025,-0.092185,-0.517462,-0.528700,-0.089885,0.217005,0.116031,-1.190926,...,0.415520,-0.061243,-0.090552,0.087959,0.024043,0.131635,0.645794,0.025367,-0.072784,0.045389
2,100033323702,0,3689.025,-0.092185,0.369017,0.157052,-0.089885,0.217005,0.410344,-0.241197,...,0.415520,-0.061243,-0.090552,1.078183,0.607950,0.131635,-0.685209,0.940320,0.261499,-0.676637
3,100048415270,0,1169.000,-0.092185,-0.123471,0.842804,-0.089885,0.217005,-0.444566,-1.277265,...,-1.597133,16.114420,-0.090552,0.418034,-0.451733,0.131635,1.477670,-0.020380,0.242397,-0.939192
4,100056526386,0,3689.025,-0.092185,0.664511,1.528556,-0.089885,0.217005,0.144060,0.881210,...,1.019316,-0.061243,-0.090552,1.738332,1.862269,0.131635,-0.685209,-0.889585,-0.087111,-0.020249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,-0.092185,-0.615960,-1.214452,-0.089885,-3.810966,0.045956,-0.845570,...,-0.590806,-0.061243,-0.090552,0.748108,-0.213845,-1.324446,-0.685209,0.505717,0.132561,-1.464301
4310,416771521620,0,3689.025,-0.092185,1.255497,-0.528700,-0.089885,0.217005,-0.598730,0.276837,...,0.415520,-0.061243,-0.090552,-0.902264,-0.862631,0.131635,-0.685209,-0.180497,-0.120539,1.555080
4311,416783315386,0,3689.025,-0.092185,-0.517462,-0.528700,-0.089885,0.217005,-0.472596,0.104159,...,-0.590806,-0.061243,-0.090552,0.017451,-0.244607,0.131635,-0.186083,-0.059783,-0.105837,-0.033068
4312,416796224310,0,3689.025,-0.092185,-0.024974,-0.528700,-0.089885,0.217005,-0.570700,0.017820,...,-0.590806,-0.061243,-0.090552,-1.562413,2.402923,0.131635,1.311295,0.162610,0.581455,1.292525


In [9]:
data_full = data_full

In [10]:
# decide on number of dicrete times:
#halflife=13.*365.25


#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [11]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [12]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    # l2_final=0.1

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)






FOLD 0:

Train C-index fold 1 :
0.8500460240828058
Test C-index fold 1 :
0.723618700667881

FOLD 1:

Train C-index fold 2 :
0.8451784053822452
Test C-index fold 2 :
0.8900081677103185

FOLD 2:

Train C-index fold 3 :
0.8321069503132741
Test C-index fold 3 :
0.6994535519125683

FOLD 3:

Train C-index fold 4 :
0.8678548329795859
Test C-index fold 4 :
0.7057380400708737

FOLD 4:

Train C-index fold 5 :
0.8550604635080663
Test C-index fold 5 :
0.8096738832556865

FOLD 5:

Train C-index fold 6 :
0.8267228278204094
Test C-index fold 6 :
0.7983172783806853

FOLD 6:

Train C-index fold 7 :
0.8555199083877887
Test C-index fold 7 :
0.7502065546681355

FOLD 7:

Train C-index fold 8 :
0.8524530499344882
Test C-index fold 8 :
0.7230980123372173

FOLD 8:

Train C-index fold 9 :
0.8739485842909608
Test C-index fold 9 :
0.8350149986364875

FOLD 9:

Train C-index fold 10 :
0.8745327571431176
Test C-index fold 10 :
0.7329707681849083

FOLD 10:

Train C-index fold 11 :
0.8542359854910819
Test C-index fo

In [13]:
# # cross validation:
# from sklearn.model_selection import StratifiedKFold
# n_folds = 10
# kf=StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
# early_stopping = EarlyStopping(monitor='loss', patience=20)

# #l2_array = np.concatenate(([0.],np.power(10.,np.arange(-6,-2))))
# l2_array = np.power(10.,np.arange(-3,2))
# grid_search_train = np.zeros((len(l2_array),n_folds))
# grid_search_test = np.zeros((len(l2_array),n_folds))
# grid_search_c_index_train = np.zeros((len(l2_array),n_folds))
# grid_search_c_index_test = np.zeros((len(l2_array),n_folds))

# for i in range(len(l2_array)):
#     #i = 0
#     j=0
#     cv_folds = kf.split(train_df.values, train_df[['event']].values)
#     for traincv, testcv in cv_folds:
#         print('i in len(l2_array)=' + str(i+1) + '/' + str(len(l2_array)))
#         print('cv fold =' + str(j+1) + '/' + str(n_folds))
#         x_train_cv = featurespace_train_df.iloc[traincv].values
#         y_train_cv = y_train[traincv]
#         x_test_cv = featurespace_train_df.iloc[testcv].values
#         y_test_cv = y_train[testcv]

#         model = Sequential()
#         #model.add(Dense(n_intervals,input_dim=x_train.shape[1],bias_initializer='zeros',kernel_regularizer=regularizers.l2(l2_array[i])))
#         model.add(Dense(hidden_layers_sizes
#                   , input_dim=featurespace_train.shape[1]
#                   , bias_initializer='zeros'
#                   , activation='relu'
#                   , kernel_regularizer=regularizers.l2(l2_array[i])
#                   ))
#         model.add(Dense(n_intervals))
#         model.add(Activation('sigmoid'))
#         model.compile(loss=nnet_survival.surv_likelihood(n_intervals)
#                       , optimizer=optimizers.Adam()) #lr=0.0001))
#         history=model.fit(x_train_cv, y_train_cv
#                     , batch_size=256
#                     , epochs=1000
#                     , callbacks=[early_stopping]
#                     , verbose=1
#                     , validation_data=(x_test_cv, y_test_cv))
#         grid_search_train[i,j] = model.evaluate(x_train_cv,y_train_cv,verbose=0)
#         grid_search_test[i,j] = model.evaluate(x_test_cv,y_test_cv,verbose=0)
              
#         #Discrimination performance
#         # y_pred is conditional prob of survival within each time interval
#         y_pred_train_cv = model.predict_proba(x_train_cv ,verbose=0)
#         yr26_surv_train_cv = nnet_pred_surv(y_pred_train_cv, breaks, 365.25*26)
#         y_pred_test_cv=model.predict_proba(x_test_cv ,verbose=0)
#         yr26_surv_test_cv = nnet_pred_surv(y_pred_test_cv, breaks, 365.25*26)
        
#         grid_search_c_index_train[i,j] = concordance_index(train_df.time.iloc[traincv], yr26_surv_train_cv, train_df.event.iloc[traincv])
#         grid_search_c_index_test[i,j] = concordance_index(train_df.time.iloc[testcv], yr26_surv_test_cv, train_df.event.iloc[testcv])
        
#         print(' ')
#         print('Looking at  i=' + str(i+1) + '/' + str(len(l2_array)) + (' and cv fold =' + str(j+1) + '/' + str(n_folds)))      
#         print('Loss Train, Loss Valid, C-index Train, C-index Test:')
#         print(grid_search_train[i,j])
#         print(grid_search_test[i,j])
#         print(grid_search_c_index_train[i,j])
#         print(grid_search_c_index_test[i,j]) 
#         print(' ')
              
#         j=j+1
        
    


# print('grid_search_train:')
# print(grid_search_train)
# print('grid_search_test:')
# print(grid_search_test)
# print(np.average(grid_search_train,axis=1))
# print(np.average(grid_search_test,axis=1))
# print(grid_search_c_index_train)
# print(grid_search_c_index_test)
#
#l2_final = l2_array[np.argmax(-np.average(grid_search_test,axis=1))]    

In [14]:

# ##########################
# # Build model with tuned hyperparam:
# l2_final=0.0001

# from numpy.random import seed

# seed(1)
# import tensorflow as tf
# import keras
# tf.random.set_seed(2)

# model = Sequential()
# model.add(Dense(hidden_layers_sizes
#                 , input_dim=featurespace_train.shape[1]
#                 , bias_initializer='zeros'
#                 , kernel_regularizer=regularizers.l2(l2_final)))
# model.add(Activation('relu'))
# model.add(Dense(n_intervals))
# model.add(Activation('sigmoid'))
# # import keras.backend.tensorflow_backend as kk

# model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
# early_stopping = EarlyStopping(monitor='loss', patience=20)
# history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
#                   , callbacks=[early_stopping]
#                   , verbose=0)


# # print loss of train and valid data:
# # print(model.evaluate(featurespace_train,y_train,verbose=0))
# # model.evaluate(featurespace_test,y_test,verbose=0)

# #Discrimination performance
# # y_pred is conditional prob of survival within each time interval
# y_pred_train = model.predict(featurespace_train,verbose=0)
# # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
# # index -1 because we are interested in -1 
# last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
# print('Train C-index fold', str(fold+1),':')
# print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

# y_pred=model.predict(featurespace_test,verbose=0)
# last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
# print('Test C-index fold', str(fold+1),':')
# print(concordance_index(test_df.time,last_yr_surv, test_df.event))

# pred_surv = np.zeros((len(test_df.event), len(eval_times)))
# col=0
# for time in eval_times:
#     pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
#     col = col+1
# pred_surv = pd.DataFrame(data = pred_surv)
# # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
# savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
# try: 
#     os.makedirs(savedir)
# except OSError:
#     if not os.path.isdir(savedir):
#         raise
# actual_fold = fold+1
# pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)




In [ ]:
!pip install scikit-survival